<a href="https://colab.research.google.com/github/YifanSong348/STATS302/blob/main/Week%202/Default.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#STATS302 HW 2 - Yifan Song
###Problem 4. Programming

In [42]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneOut, KFold
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.utils import resample
from sklearn import metrics

# Load StatsModels API
# Note that if we wish to use R-style formulas, then we would use the StatsModels Formula API
import statsmodels.api as sm
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import files
import io

In [43]:
# import dataset
uploaded = files.upload()
data = pd.read_csv(io.BytesIO(uploaded['Default.csv']))
data = data.drop(data.columns[0], axis=1)
print(data.shape)
data.head()

Saving Default.csv to Default (2).csv
(10000, 4)


,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879


In [44]:
data.columns

Index(['default', 'student', 'balance', 'income'], dtype='object')

In [45]:
data.describe()

,balance,income
count,10000.000000,10000.000000
mean,835.374886,33516.981876
std,483.714985,13336.639563
min,0.000000,771.967729
25%,481.731105,21340.462903
50%,823.636973,34552.644802
75%,1166.308386,43807.729272
max,2654.322576,73554.233495


# a.

**Approach 1**

We could use `LogisticRegression` from `scikit-learn`. But let's use `statsmodel` for simplicity.

First, we need to replace `Yes` and `No` with boolean variables or {1,0}. This can be done using `replace` or `map`.
Also, to produce the same result as in the text, we need to put income in thousand-dollars.

In [46]:
data1 = data.replace({"default": {"Yes": True, "No": False}, "student": {"Yes": 1., "No": 0.}})
# data1["income"] = data1["income"]/1000.
data1.head()

,default,student,balance,income
0,False,0.0,729.526495,44361.625074
1,False,1.0,817.180407,12106.134700
2,False,0.0,1073.549164,31767.138947
3,False,0.0,529.250605,35704.493935
4,False,0.0,785.655883,38463.495879


Note that in `statsmodel.Logit`, by default, there is no constant in the logit. We need to add a constant using `add_constant`.

Let's look at the simple logistic regression on `student`.

In [47]:
X = data1[["balance", "income"]]
X = sm.add_constant(X)
y = data1["default"]
results = sm.Logit(y,X).fit()
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.078948
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Tue, 06 Apr 2021   Pseudo R-squ.:                  0.4594
Time:                        06:23:32   Log-Likelihood:                -789.48
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                4.541e-292
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.5405      0.435    -26.544      0.000     -12.393     -10.688
balance        0.0056      0

**Approach 2**

In [48]:
X = data1[["balance", "income"]]
y = data1["default"]
clf = LogisticRegression(penalty = "none", solver = "lbfgs")
clf.fit(X, y)
clf.coef_

array([[5.64710291e-03, 2.08089921e-05]])

# b.

I will use a 75-25 split when dividing my data into a training set and a validation set. To split the data into a training set and a validation set, we use train_test_split(). Note that we passed a value to the random_state argument to have reproducible results. After fitting the logistic regression model using the training observations, scikit-learn performs steps 3 and 4 (obtaining predictions using a posterior probability threshold of 0.5 and then computing the validation set accuracy) in a single step via the score() function.

In [49]:
# Split the sample set into a training set and a validation set.
X_train, X_test, y_train, y_test = train_test_split(data1[["balance", "income"]], data1["default"], test_size = 0.25, random_state = 312)
# Fit a multiple logistic regression model using only the training observations.
clf = LogisticRegression(penalty = "none", solver = "lbfgs")
clf.fit(X_train, y_train)
# Compute the validation set error
1 - clf.score(X_test, y_test)

0.02839999999999998

For this train-test split, we had a validation set error of 0.0284, or 2.84% of the observations in the validation set that are misclassified.

If we wished to make predictions using a posterior probability threshold other than 0.5, we would want to use the predict_proba() function to obtain the posterior probabilities of each class for each observation in the validation set.

In [50]:
print(clf.predict_proba(X_test))

[[9.99510227e-01 4.89772691e-04]
 [9.98863554e-01 1.13644553e-03]
 [9.87071086e-01 1.29289144e-02]
 ...
 [9.99967740e-01 3.22599080e-05]
 [9.99987918e-01 1.20816251e-05]
 [9.65916850e-01 3.40831497e-02]]


In [51]:
y_pred=clf.predict(X_test)
print(y_pred)
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
print(cnf_matrix)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

[False False False ... False False False]
[[2394   14]
 [  57   35]]
Accuracy: 0.9716
Precision: 0.7142857142857143
Recall: 0.3804347826086957


# c.

In [52]:
# Split the sample set into a training set and a validation set.
X_train, X_test, y_train, y_test = train_test_split(data1[["balance", "income"]], data1["default"], test_size = 0.25, random_state = 456)
# Fit a multiple logistic regression model using only the training observations.
clf = LogisticRegression(penalty = "none", solver = "lbfgs")
clf.fit(X_train, y_train)
# Compute the validation set error
1 - clf.score(X_test, y_test)

0.03200000000000003

In [53]:
# Split the sample set into a training set and a validation set.
X_train, X_test, y_train, y_test = train_test_split(data1[["balance", "income"]], data1["default"], test_size = 0.25, random_state = 789)
# Fit a multiple logistic regression model using only the training observations.
clf = LogisticRegression(penalty = "none", solver = "lbfgs")
clf.fit(X_train, y_train)
# Compute the validation set error
1 - clf.score(X_test, y_test)

0.034399999999999986

In [54]:
# Split the sample set into a training set and a validation set.
X_train, X_test, y_train, y_test = train_test_split(data1[["balance", "income"]], data1["default"], test_size = 0.25, random_state = 12321)
# Fit a multiple logistic regression model using only the training observations.
clf = LogisticRegression(penalty = "none", solver = "lbfgs")
clf.fit(X_train, y_train)
# Compute the validation set error
1 - clf.score(X_test, y_test)

0.03320000000000001

In [55]:
(0.02839999999999998 + (0.03200000000000003 + 0.034399999999999986 + 0.03320000000000001)/3)/2

0.030799999999999994

In [56]:
(data["default"] != "No").mean()

0.0333

Using three different 75-25 splits of the data, the validation set error remained fairly consistent. The average error between this part and Part 2 was 0.0307, and errors were all within about 20% of each other. However, these test errors are all fairly close to the error one would achieve with a naive strategy of just saying that nobody will default. The average of the test errors is only about 5% less than the error from using the naive strategy.

# d.

In [57]:
np.random.seed(312)
# with_student = {}
# without_student = {}

# # Create two classifier pipelines
# # with_student takes the student variable and encodes it using one hot encoding, passes through income, balance
# # without_student drops the student variable and only passes through income and balance
# categorical_features = ["student"]
# categorical_transformer = Pipeline([("onehot", OneHotEncoder(drop = "first"))])
# numerical_features = ["income", "balance"]
# with_student_preprocessor = ColumnTransformer([("cat", categorical_transformer, categorical_features),
#                                  ("num", "passthrough", numerical_features)])
# with_student_clf = Pipeline([("preprocessor", with_student_preprocessor), 
#                 ("classifier", LogisticRegression(penalty = "none", solver = "lbfgs"))])
# without_student_preprocessor = ColumnTransformer([("num", "passthrough", numerical_features)])
# without_student_clf = Pipeline([("preprocessor", without_student_preprocessor), 
                # ("classifier", LogisticRegression(penalty = "none", solver = "lbfgs"))])

# Loop through 50 train-test splits to compute average difference in error rate
for i in range(50):
    # Split the data in to training and test sets
    X_train, X_test, y_train, y_test = train_test_split(data, data["default"], test_size = 0.25)
    # Fit classifier which includes student variable and compute validation set error
    with_student_clf.fit(X_train, y_train)
    with_student[i] = 1 - with_student_clf.score(X_test, y_test)
    # Fit classifier which excludes student variable and compute validation set error
    without_student_clf.fit(X_train, y_train)
    without_student[i] = 1 - without_student_clf.score(X_test, y_test)
errors = pd.DataFrame({"with_student": with_student, "without_student": without_student})
errors["difference"] = errors["with_student"] - errors["without_student"]
errors["difference"].mean()

0.0021280000000000075

Looping through 50 train-test splits and comparing the error rates between the logistic regression model predicting default using income, balance, and student and the logistic regression model predicting default using just income and balance for each split, we see that, on average, including a dummy variable for student does not lead to a reduction in the test error rate. In fact, on average it resulted in a very slight increase in the test error rate.

In [58]:
# Split the sample set into a training set and a validation set.
X_train, X_test, y_train, y_test = train_test_split(data1[["balance", "income"]], data1["default"], test_size = 0.25, random_state = 789)
# Fit a multiple logistic regression model using only the training observations.
clf = LogisticRegression(penalty = "none", solver = "lbfgs")
clf.fit(X_train, y_train)
# Compute the validation set error
1 - clf.score(X_test, y_test)

0.034399999999999986

In [59]:
# Split the sample set into a training set and a validation set.
X_train, X_test, y_train, y_test = train_test_split(data1[["balance", "income", "student"]], data1["default"], test_size = 0.25, random_state = 789)
# Fit a multiple logistic regression model using only the training observations.
clf = LogisticRegression(penalty = "none", solver = "lbfgs")
clf.fit(X_train, y_train)
# Compute the validation set error
1 - clf.score(X_test, y_test)

0.03639999999999999